In [ ]:
import cv2
import glob
import itertools
import os
from pathlib import Path

In [ ]:
input_filespecs = [
    '../../data/*.mpg',
#     '../../data/virat/*/*.mpg',
#     '../../data/virat/*/*.mp4',
]
#size = (2048, 1080)  # 2K
size = (1280, 720) # 720p
#size = None
fps = 15

In [ ]:
def transcode(input_filename, quality=50, skip_if_exists=True, size=None, fps=None):
    output_dir = Path(input_filename).with_suffix('')
    print(f'{input_filename} -> {output_dir}')
    if skip_if_exists and output_dir.exists():
        print('Output directory already exists. Skipping.')
        return
    vidcap = cv2.VideoCapture(input_filename)
    output_dir.mkdir(exist_ok=True)
    frame_number = 0
    while True:
        success, image = vidcap.read()
        if not success:
            break
        drop = False
        if fps is not None:
            input_ms = vidcap.get(cv2.CAP_PROP_POS_MSEC)
            output_ms = frame_number * 1000.0 / fps
            drop = input_ms < output_ms
        if not drop:
            if size:
                image = cv2.resize(image, size, interpolation=cv2.INTER_AREA)
            output_filename = output_dir / ('%08d.jpg' % frame_number)
            print(f'{output_filename}', end='\r')
            cv2.imwrite(str(output_filename), image, [int(cv2.IMWRITE_JPEG_QUALITY), quality])
            frame_number += 1
    print('')

In [ ]:
%%time
for input_filespec in input_filespecs:
    for input_filename in sorted(glob.glob(input_filespec)):
        transcode(input_filename, skip_if_exists=True, size=size, fps=fps)